In [1]:
# Virtual Environment
# All the units are in SI system
import numpy as np
# 

# Virtual Environment
# All the units are in SI system
import numpy as np
# 

def Matrial_on_the_road(Air_temp, Suraface_temp, Precipitation, wind_vel,initial_Snow, initial_water, initial_Salt, initial_ice, plowing):
    # General assumptions:
    Dataset_hour_inc = 1 # *3 had applied in the dataset
    f_t = 0.9 #times when the road is covered with moving vehicle
    g_t = 1 - f_t
    
    # print ("Air_temp, Suraface_temp, Precipitation, wind_vel,initial_Snow, initial_water, initial_Salt, initial_ice, plowing")
    # print (Air_temp, Suraface_temp, Precipitation, wind_vel,initial_Snow, initial_water, initial_Salt, initial_ice, plowing)
    
    # convert Precipitation to snow or rain based on the air temp
    if Air_temp < .5:
        Snowfall_amount = Precipitation
        Rainfall_amount = 0
    else:
        Rainfall_amount = Precipitation
        Snowfall_amount = 0
    
    ###### Heat Ballance ##########################################################################################################
    # Q_csp : Flux of pavement heat
    V_wis = initial_ice + initial_Snow + Snowfall_amount # Volume of WIS layer (m3*m-2 = m) depth of material on the road
    V_ps = 50e-3 # thikness of pavement surface, generally 25 to 55 mm, here we assume 50mm
    Lambda_wis = 0.8 # thermal conductivity of WIS layer. Assume that is compacted snow 
    Lambda_P = 1.5 # Thermal conductivity of pavemetn. 0.8 to 2
    T_wis = (Air_temp + Suraface_temp)/2 #Temp of wis layer: assumption 
    Q_csp = 1/((V_wis/(2*Lambda_wis))+(V_ps/(2*Lambda_P)))*(Suraface_temp - T_wis)
    
     # Q_rn : Flux of net radient heat
    q_rld = 20 #Since we are modeling the winter stroms, the sky radiation shouldn't be high 0-200
    q_rlu = 0.97 * 5.64e-8 * (T_wis + 273.15)**4
    q_rsd = 20 #Since we are modeling the winter stroms, the sky radiation shouldn't be high 0-300
    q_rsu = 0.3 * q_rsd
    Q_rn = f_t*q_rld + q_rlu + f_t*q_rsd - q_rsu
    
    # Q_sn Flux of net radiant heat
    q_sa = (10e4 * wind_vel**0.7 +2.2)*(T_wis - Air_temp)#snesable heat flux due to wind
    q_sf = (4.184* (initial_water + Rainfall_amount) + 2.108 * (initial_ice  + initial_Snow + Snowfall_amount))# Sensable heat flux due to rainfall and snowfall
    q_sr = 0 #Sensible heat flux of drainage due to road gradient assume = 0
    q_sv = 4.184 * T_wis * 1000 * 0.5 * 0.005/3600 * 0.15#sensible heat flux of water dispersion due to passing vehicle
    Q_sn = q_sa + f_t*q_sf + q_sr + g_t * q_sv
    
    # Q_ln flux of net latent heat
    # m_wi * L_wi is not considered here, we will change the q_net relation with M_wi later
    m_il = 0 # sublimation flux
    L_i = 2838 #kJkg-1 latent heat of sublimation
    m_wl = 0 # flux of evaporation and condensation
    L_w = 2260 #kJkg-1 latent heat of evaporation and condensation
    m_sl = 3.34e-5 # dissolving flux (0 - 6.67e-5) if salting
    L_s = -66.4 # latent heat of dissolution of salt 
    Q_ln = m_il*L_i + m_wl*L_w + m_sl*L_s
    
    # Q_vn flux of net vehicle heat
    Q_vn = 100 #assumption according to Fujimoto and other citations
    
    # Q_net
    Q_net = Q_csp + Q_rn + Q_sn + Q_ln + Q_vn
    #################################################################################################################################
    #*******************************************************************************************************************************#
    
    ## Material creation $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    # Ice 
    Ice_final = initial_ice 
    
    # Snowfall 
    m_snowf = Snowfall_amount * Dataset_hour_inc * 100 # 100 kg/m3
    Snow_creat = m_snowf * f_t
    Snow_final = Snow_creat + initial_Snow
    # plowing  $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    if plowing == 1:
        Snow_final *= 0.1
        
    # or rainfall
    m_wf = Rainfall_amount * Dataset_hour_inc * 1000 # 1000 kg/m3
    Rain_creat = m_wf * f_t
    Water_final = Rain_creat + initial_water
    # before mesuring the water to ice, we have to consider the water dispersion
    # Water dispersion
    # Assume we have good drainage system
    Water_final = Water_final * 0.95
    
    #Salt
    # Salt_final = initial_Salt * 0.5# Maybe need conversion
    
    ### Material Conversion $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    ## Snow to ice
    Snow_To_Ice_rate = (0.1 if initial_Snow >0 else 0) # Assumption: conversion applied to initial snow 
    Ice_final += Snow_To_Ice_rate * initial_Snow
    
    ## water to ice or vise versa *
    L_wi = 334 #kJkg-1 latent heat of melting and freezing
    Water_ice_conv_mass = abs(Q_net*1000/(L_wi*999)) if initial_Snow >0 or initial_water>0 or initial_ice>0 else 0
    # We will assume that the water freezing point is just a function of salt concentration
    T_frezzing = -0.025 * initial_Salt - 0.5 #0.04 * initial_Salt + 1#
    #T_frezzing = (T_frezzing -32)*5/9 
    # print("TF",T_frezzing, "initial_Salt", initial_Salt, "T_wis", T_wis  )
    
    if T_wis >= T_frezzing:
        STATUS = "Melting"
        # print("###############################Melting")  # Melting
        stat_sign = 1
        Ice_final = Ice_final - min(Water_ice_conv_mass, Ice_final)  
        Water_final = Water_final + min(Water_ice_conv_mass, Ice_final) 
    elif T_wis < T_frezzing and initial_Salt ==0:
        STATUS = "Freezing"
        # print("###############################Freezing")  # Freezing
        Ice_final = Ice_final + min(Water_ice_conv_mass, Water_final)  
        Water_final = Water_final - min(Water_ice_conv_mass, Water_final) 
    elif T_wis < T_frezzing and initial_Salt >0:
        STATUS = "Freezing slowly"
        # print("###############################Freezing slowly")  # Freezing
        Ice_final = 0.2*(Ice_final + min(Water_ice_conv_mass, Water_final))
        Water_final = Water_final - min(Water_ice_conv_mass, Water_final)
 
    # Snow to water
    if STATUS == "Melting":
        Water_final = Water_final + stat_sign * Water_ice_conv_mass
        Water_final = Water_final * 0.05 #disperssed
        Snow_final = Snow_final - stat_sign * Water_ice_conv_mass
        # print ("Water_ice_conv_mass", Water_ice_conv_mass)
    ## Material dispersion $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    # Salt dispersion: depands on STATUS
    if STATUS == "Melting":
        Salt_disp_rate = 0.9
    if STATUS == "Freezing" or STATUS == "Freezing slowly":
        Salt_disp_rate = 0.98
    Salt_final = initial_Salt * Salt_disp_rate
    
    
    # Check each variable and set to zero if negative
    Water_final = max(Water_final, 0)
    Salt_final = max(Salt_final, 0)
    Ice_final = max(Ice_final, 0)
    Snow_final = max(Snow_final, 0)
    
    
    # print ("Water_final, Salt_final, Ice_final, Snow_final", Water_final, Salt_final, Ice_final, Snow_final)
    return Water_final, Salt_final, Ice_final, Snow_final
print ("done")


done


In [2]:
import numpy as np
import pandas as pd
from collections import deque
import random
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
import tensorflow.keras.backend as K

class DQNAgent:
    def __init__(self, state_size, action_size, model_path=None):
        self.state_size = state_size
        self.action_size = action_size
        self.action_list_size = action_size
        self.memory = deque(maxlen=10000) #
        self.gamma = 0.99  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.999
        self.learning_rate = 0.00001 #<<<<<<<<<< 
        self.Number_ofExploitation = 0
        self.Number_ofExploration = 0
        if model_path:
            self.model = load_model(model_path)
            print("Model loaded from", model_path)
        else:
            self.model = self._build_model()
            print("New model initialized")

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
         ## Input layer and first hidden layer
        model.add(Dense(48, input_dim=total_state_size, activation='relu'))
        ## Second hidden layer
        # model.add(Dense(96, activation='relu'))
        # ## Adding Dropout to prevent overfitting
        # model.add(Dropout(0.2))
        
        ## Third hidden layer
        #model.add(Dense(512, activation='relu'))
        ## Adding Dropout to prevent overfitting
        #model.add(Dropout(0.2))
        
        ## Fourth hidden layer
        #model.add(Dense(256, activation='relu'))

        model.add(Dense(124, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(124, activation='relu'))
        model.add(Dense(self.action_list_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        

    def act(self, state):
        if False: #np.random.rand() <= self.epsilon:
            # Biased action selection
            actions_probability = [0.4, 0.2, 0.2, 0.2]  # # Example: 40% do nothing, 20% for each other action
            self.Number_ofExploration +=1
            return np.random.choice(np.arange(self.action_size), p=actions_probability)
        else:
            act_values = self.model.predict(state)
            self.Number_ofExploitation +=1
            return np.argmax(act_values[0])
    
    def predict_action(self, state):
        """Selects the action with the highest Q-value for the given state."""
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
                
    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
            
print ("done")

2024-04-07 22:11:52.244383: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 22:11:52.285950: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-07 22:11:52.286017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-07 22:11:52.286953: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-07 22:11:52.296187: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-07 22:11:52.297039: I tensorflow/core/platform/cpu_feature_guard.cc:1

done


In [3]:

import numpy as np

class RoadEnv:
    def __init__(self, dataset, unique_window_ids):
        self.dataset = dataset
        self.current_state = None
        self.unique_window_ids = unique_window_ids 
        self.current_episode_index = 0
        # self.action_costs = {0: 0, 1: 1, 2: 6, 3: 5}
        self.reset()
#     def simulate_no_intervention(self, window_data):
#         no_intervention_results = []
#         initial_conditions = {'initial_Snow': 0, 'initial_water': 0, 'initial_Salt': 0, 'initial_ice': 0, 'plowing':0}
#         for index, row in window_data.iterrows():
#             # Simulate step without interventions
#             Air_temp = row['Air TemperatureC']
#             Surface_temp = row['Surface TemperatureC']
#             Precipitation = row['Precipitation Intensitym/3h'] 
#             wind_vel = row['Wind Speed (act)m/s']
#             water, salt, ice, snow = Matrial_on_the_road(Air_temp, Surface_temp, Precipitation, wind_vel, **initial_conditions)
#             # print ("water, salt, ice, snow", water, salt, ice, snow)
#             # Update initial conditions for the next step
#             initial_conditions.update({'initial_Snow': snow, 'initial_ice': ice, 'initial_water': water, 'initial_Salt': salt, 'plowing': 0})
            
#             # Append results for no intervention forecast
#             no_intervention_results.append((water, salt, ice, snow))
#             # print("no_intervention_results",no_intervention_results)
#         return no_intervention_results
    # def reset(self):
    #     # Reset the environment to the initial state of a new episode
    #     self.current_state = self.dataset.iloc[0].copy()
    #     self.plowing_count = 0
    #     self.salting_count = 0
    #     
    #     return self._get_state()
    def reset(self, episode_index=0):
        
        if self.current_state is not None:
            # Initialize the state properties if current_state is properly set
            self.current_state['initial_Snow'] = 0
            self.current_state['initial_water'] = 0
            self.current_state['initial_Salt'] = 0
            self.current_state['initial_ice'] = 0
        # print ("RESET self.current_state" )
        # print (self.current_state)
        # Calculate the start index of the episode in the dataset
        self.start_index = episode_index * 10  # Assuming each episode has 10 states
        self.current_state_index = self.start_index
        window_id = self.unique_window_ids[episode_index]
        #print ("window_id", window_id)
        # Check if the calculated start index is within the bounds of the dataset
        if self.start_index >= 0 and self.start_index < len(self.dataset):
            self.current_state = self.dataset.iloc[self.start_index].copy()
        else:
            # Fallback to the first row if the calculated index is out of bounds
            self.current_state = self.dataset.iloc[0].copy()
            print(f"Warning: Episode index {episode_index} out of bounds, falling back to initial state.")
    
        # Reset the action counts for the new episode
        self.plowing_count = 0
        self.salting_count = 0
        
        # Reset environment to the start of a new episode, with modifications to include no intervention forecast
        window_id = unique_window_ids[episode_index]
        episode_data = self.dataset[self.dataset['Window_ID'] == window_id]
        self.Lowest_temp = min(min(episode_data['Surface TemperatureC']), min(episode_data['Air TemperatureC']))
        #print ("self.Lowest_temp", self.Lowest_temp)
        # self.no_intervention_forecast = self.simulate_no_intervention(episode_data)
        # print ("self.no_intervention_forecast", self.no_intervention_forecast)
        # Return the initial state of the new episode
        return self._get_state()

    def step(self, action):
        reward = 0#20000
        
        # Check if the action is allowed based on the resource limitations
        if action == 1 and self.plowing_count >= 3:  # Plowing limit reached
            reward = -1000  # Force 'no action'
        elif action == 3 and self.salting_count >= 2:  # Salting limit reached
            reward = -1000  # Force 'no action'
        elif action == 2:  # Plowing and salting
            if self.plowing_count >= 3 and self.salting_count >= 2:
                reward = -2000  # If either limit is reached, force 'no action'
            elif self.plowing_count >= 3:
                reward = -1000
            elif self.salting_count >= 2:
                reward = -1000
    
        # Update action counters
        if action == 1:  # Plowing
            self.plowing_count += 1
        elif action == 3:  # Salting
            self.salting_count += 1
        elif action == 2:  # Plowing and salting
            self.plowing_count += 1
            self.salting_count += 1
        # Apply the chosen action to the environment and update the state
        # print ("action", action)
        
        water, salt, ice, snow = self._apply_action(action)
        # print (" water, salt, ice, snow", water, salt, ice, snow)
        # Update the current state with the new values
        self.current_state['initial_Snow'] = snow
        self.current_state['initial_water'] = water
        self.current_state['initial_Salt'] = salt
        self.current_state['initial_ice'] = ice
        #print ("self.current_state &&&&&Step", self.current_state)
        # Calculate the reward
        reward += self._calculate_reward(action, ice, snow, salt)
        # print ("reward",reward)
        # Move to the next row in the dataset, simulating time progression
        
        # Dynamically determine if it's the last state in the episode
        current_episode_index = self.dataset.index.get_loc(self.current_state.name)
        # print ("current_episode_index *********************   ", current_episode_index)
        #episode_start_index = current_episode_index - (current_episode_index % 10)  # Assuming episodes start at indices 0, 10, 20, ...
        #next_index = current_episode_index + 1
        #is_last_state_in_episode = (next_index % 10 == 0) or (next_index >= len(self.dataset))
        is_last_state_in_episode = (self.current_state_index - self.start_index ==9)
        if is_last_state_in_episode:
            done = True
            
        else:
            self.current_state_index = self.current_state_index + 1 
            
            next_state = self.dataset.iloc[self.current_state_index].copy()
            for col in ['MeasureTime', 'Window_ID', 'Air TemperatureC', 'Surface TemperatureC', 'Wind Speed (act)m/s', 'Precipitation Intensitym/3h']:
                  self.current_state[col] = next_state[col]
            done = False
        #print ("self.current_state_index", self.current_state_index, "self.start_index", self.start_index)
        # print ("SETP self.current_state" )
        # print (self.current_state)
        
        return self._get_state(), reward, done

    def _get_state(self):
        # Extract the current state from the dataset row
        current_state_values = [self.current_state[col] for col in ['Air TemperatureC', 'Surface TemperatureC', 
                                                                     'Wind Speed (act)m/s', 'Precipitation Intensitym/3h', 
                                                                     'initial_Snow', 'initial_water', 'initial_Salt', 'initial_ice']]
        
        # Prepare no intervention forecast data for the entire episode
        # Assuming no_intervention_forecast is a list of tuples for the entire episode at this point
        #no_intervention_forecast_flat = [value for forecast in self.no_intervention_forecast for value in forecast]
    
        # Combine the current state values with the no intervention forecast
        # Here, we ensure the state_with_forecast includes the no intervention data for the entire episode
        state_ = np.array(current_state_values)# + no_intervention_forecast_flat)
        # print ("state_with_forecast", state_with_forecast)
        return state_

    def _apply_action(self, action):
        # Apply the chosen action and update the road conditions
        # For simplicity, this function assumes the existence of a function named 'Matrial_on_the_road'
        # that calculates the final amounts of water, salt, ice, and snow

        air_temp = self.current_state['Air TemperatureC']
        surface_temp = self.current_state['Surface TemperatureC']
        precipitation = self.current_state['Precipitation Intensitym/3h']
        wind_vel = self.current_state['Wind Speed (act)m/s']
        initial_snow = self.current_state['initial_Snow']
        initial_water = self.current_state['initial_water']
        initial_salt = self.current_state['initial_Salt']
        initial_ice = self.current_state['initial_ice']
        plowing  = 0
        #print ("self.current_state _apply_action", self.current_state)
        if action == 1 or action == 2:  # Plowing or Plowing and Salting
            plowing  = 1
            

        if action == 2 or action == 3:  # Salting or Plowing and Salting
            # Measuring salt amount based on the Ruled-Base model
                Salt_amount = -25 * self.Lowest_temp + 25 #-32.027 * self.Lowest_temp + 79.24
                # print ("Salt_amount: ", Salt_amount)
                if Salt_amount < 0: Salt_amount = 0
                if Salt_amount > 400: Salt_amount = 400  # here salt unit is lb/mile/lane
                initial_salt = Salt_amount  # Assuming a constant amount of salt is used

        water_final, salt_final, ice_final, snow_final = Matrial_on_the_road(
            air_temp, surface_temp, precipitation, wind_vel,
            initial_snow, initial_water, initial_salt, initial_ice , plowing
        )
        # print (f"action {action}, water_final {water_final}, salt_final {salt_final}, ice_final {ice_final}, snow_final {snow_final}" )
        return water_final, salt_final, ice_final, snow_final

    def _calculate_reward(self, action, ice, snow, salt):
        # Calculate the reward based on the reduction of ice and snow, and the cost of the action
        # cost = self.action_costs[action]
        cost = 0
        if action == 1: # Plowing
            cost = 12#4.3
        if action == 2: # Plowing and saltin
            cost = 12 + 0.07*salt
        if action == 3:
            cost =  0.07*salt#4.5 + 0.07*salt
        # Penalties
        # Convert kg/m2 to inch of snow
        snow_convert = snow /2.5 #100kg/m3
        ice_convert = ice /6/2.5 #600 kg/m3
        
        #function correlating snow accumulation depth to decreased traffic speed
        f_spped_snow = 0
        if snow_convert <= 0.1: f_spped_snow = 0
        elif snow_convert <0.5 and  snow_convert > 0.1: f_spped_snow = snow_convert * 0.2 * 100
        elif snow_convert >= 0.5: f_spped_snow = snow_convert * 0.15 * 100
        
        f_spped_ice = 0
        if ice_convert <= 0.05: f_spped_ice = 0
        elif ice_convert > 0.05: f_spped_ice = 5 * ice_convert
        
        # Cost per 3h/mile of no maintence is 250$
        Cost_speed = min(250 , max(f_spped_ice* 250, f_spped_snow * 250))
        
        #function correlating snow accumulation depth to accident probability
        f_accident_snow = 0
        if snow_convert <= 0.1: f_accident_snow = 0
        elif snow_convert <5 and  snow_convert > 0.1: f_accident_snow = (snow_convert - 0.1)/(4.9)
        elif snow_convert >= 0.5: f_accident_snow = 1
        
        f_accident_ice = 0
        if ice_convert <= 0.05: f_accident_ice = 0
        elif ice_convert > 0.05 and ice_convert< 0.5 : f_accident_ice = (ice_convert-0.05)/(0.45)
        elif ice_convert >=0.5 : f_accident_ice = 1
        
        
        
        # •	Cost accident is the cost associated with accidents
        Cost_accident = f_accident_snow * 25 + f_accident_ice * 17
            
            
#         print ("Cost_accident", Cost_accident)  
#         print ("Salt", salt)
         
        reward = - cost - Cost_speed - Cost_accident
        
        return reward
    
print ("done")

done


In [ ]:
#Training
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output

# Assuming the DQNAgent and RoadEnv classes are defined as in your previous code

# Load and preprocess dataset
file_path = 'Modified_DataSetV8_ConvertedUnits.csv'
dataset = pd.read_csv(file_path)
for col in ['initial_Snow', 'initial_water', 'initial_Salt', 'initial_ice']:
    if col not in dataset.columns:
        dataset[col] = 0

# Initialize environment and agent


action_size = 4
no_intervention_data_size = 0#10 * 4  # 10 future states * 4 variables per state
current_state_size = 8  # Current state variables
total_state_size = current_state_size + no_intervention_data_size

#agent = DQNAgent(total_state_size, action_size)
agent = DQNAgent(total_state_size, action_size, model_path='initial_55k_60k_Condor.h5')
global_episode_num = 0 

# Split dataset into training and validation
unique_window_ids = dataset['Window_ID'].unique()
env = RoadEnv(dataset, unique_window_ids)
total_training_windows = len(unique_window_ids)
print(f"Total training window IDs: {total_training_windows}")

# # Use only the first 100 episodes for training, excluding the last 10 for validation
# train_window_ids = unique_window_ids[:100]  # Adjusted to use only the first 100 episodes
validation_window_ids = unique_window_ids[-20:]  # Last 10 for validation

# Current_training_windows = len(train_window_ids)
# print(f"Current training window IDs: {Current_training_windows}")

# Create models directory
os.makedirs('models', exist_ok=True)

# Log file path
log_file_path = 'validation_log_simpleModel_Trial_2.txt'

# Training
batch_size = 16  #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
def run_episode_simulation(agent, current_episode_data, state_size, episode_num):
    best_trial_reward = -np.inf
    best_trial_data = None

    # Initial simulation setup
    #for trial in range(N_TRIALS):
        # print(f"Trial {trial}")
    trial_data = {'states': [], 'actions': [], 'rewards': [], 'next_states': [], 'dones': []}
    state = env.reset(episode_num)
        # print ("state",state)
        #cumulative_reward = 0
        # print ("current_episode_data",current_episode_data)
    i = 0
    for index, row in current_episode_data.iterrows():
        state = np.reshape(state, [1, total_state_size])
        # print ("state", state)
        if  i == 0 or i ==1:
            action = 0
        else:
            action = agent.act(state)
        i +=1
        next_state, reward, done = env.step(action)
        # print ("next_state",next_state)
        next_state = np.reshape(next_state, [1, state_size])

        # # Save trial data
        trial_data['states'].append(state)
        trial_data['actions'].append(action)
        trial_data['rewards'].append(reward)
        trial_data['next_states'].append(next_state)
        trial_data['dones'].append(done)

        state = next_state
        # cumulative_reward += reward

        # if cumulative_reward > best_trial_reward:
        #     best_trial_reward = cumulative_reward
    best_trial_data = trial_data
        #     # print ("cumulative_reward", cumulative_reward)
        #     # print ("best_trial_reward", best_trial_reward)
        # # print ("best_trial_data", best_trial_data)
    return best_trial_data
print ("len(unique_window_ids)", len(unique_window_ids))
N_TRIALS = 2000  # Number of trials to run for each episode
for batch_start in range(global_episode_num, len(unique_window_ids), N_TRIALS):
    batch_end = min(batch_start + N_TRIALS, len(unique_window_ids))
    print (f"batch_start {batch_start}, batch_end {batch_end}")
    train_window_ids = unique_window_ids[batch_start:batch_end]
    
    for i, window_id in enumerate(train_window_ids):
        print ("global_episode_num", global_episode_num)
        Current_training_windows = len(train_window_ids)
        print(f"****************************************Processing Window ID {window_id} ({i+1}/{Current_training_windows})")
        current_episode_data = dataset[dataset['Window_ID'] == window_id]
        # print ("current_episode_data", current_episode_data)
        # Run episode simulation and get the best trial data
        # print ("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii", i)
        best_trial_data = run_episode_simulation(agent, current_episode_data, total_state_size, episode_num = global_episode_num)
        #Adjust learning rate
        #agent.adjust_learning_rate(global_episode_num)

        # Increment the global episode counter after processing each episode
        global_episode_num += 1
        # Store the best trial's outcomes in memory
        for state, action, reward, next_state, done in zip(best_trial_data['states'], best_trial_data['actions'], best_trial_data['rewards'], best_trial_data['next_states'], best_trial_data['dones']):
            #print (f" state= {state} .., action={action} .., reward={reward} .., next_state = {next_state} ..,done={done}")
            #print (f" action={action} .., reward={reward}")
        
            agent.remember(state, action, reward, next_state, done)
            # Clear the output
            clear_output(wait=True)
        print ("len(agent.memory)", len(agent.memory), "batch_size", batch_size)

        if len(agent.memory) % batch_size ==0:
            agent.replay(batch_size)

    # Clear the output
    clear_output(wait=True)
    # Save the trained model
    model_name = f'models/Simple_dqn_model_episodes_{batch_start}_{batch_end}.h5'
    agent.model.save(model_name)
    validation_episode_Number = 43057
    with open(log_file_path, 'a') as log_file:
        log_file.write(f"Validation for episodes {batch_start} to {batch_end} , # Exploration: {agent.Number_ofExploration}, # Exploitation: {agent.Number_ofExploitation}  \n")
        for window_id in validation_window_ids:
            current_episode_data = dataset[dataset['Window_ID'] == window_id]
            state = env.reset(validation_episode_Number)  # Reset environment and action counters
            validation_episode_Number +=1
            episode_actions = []
            episode_info = []
            cumulative_reward = 0

            for index, row in current_episode_data.iterrows():
                state = np.reshape(state, [1, total_state_size])

                # Use predict_action to exploit the learned policy
                action = agent.predict_action(state)

                next_state, reward, done = env.step(action)
                next_state = np.reshape(next_state, [1, total_state_size])
                cumulative_reward += reward

                episode_actions.append(action)
                episode_info.append({
                    'state': state.flatten().tolist(),
                    'action': action,
                    'reward': reward,
                    'next_state': next_state.flatten().tolist(),
                    'cumulative_reward': cumulative_reward,
                    'done': done
                })

                state = next_state
                if done:
                    break

            # print(f"\nEpisode for Window ID: {window_id}")
            # print("Actions taken:", episode_actions)
            # print("Cumulative Reward:", cumulative_reward)
            for info in episode_info:
                print(info)
            # Log the actions for each episode
            log_file.write(f"Episode for Window ID: {window_id}, Actions taken: {episode_actions}, Cumulative Reward: {cumulative_reward}\n")

        print("Training and validation complete for this batch.")
print ("Training complete")



1/1 [==============================] - 0s 13ms/step
{'state': [-2.47, -6.74, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'action': 0, 'reward': 0, 'next_state': [-4.38, -7.44, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'cumulative_reward': 0, 'done': False}
{'state': [-4.38, -7.44, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'action': 0, 'reward': 0, 'next_state': [-4.62, -6.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'cumulative_reward': 0, 'done': False}
{'state': [-4.62, -6.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'action': 0, 'reward': 0, 'next_state': [-3.95, -5.84, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'cumulative_reward': 0, 'done': False}
{'state': [-3.95, -5.84, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'action': 0, 'reward': 0, 'next_state': [-0.2, 2.66, 0.0, 0.00457, 0.0, 0.0, 0.0, 0.0], 'cumulative_reward': 0, 'done': False}
{'state': [-0.2, 2.66, 0.0, 0.00457, 0.0, 0.0, 0.0, 0.0], 'action': 2, 'reward': -25.293, 'next_state': [3.27, 3.76, 0.0, 0.00762, 0.04113, 0.0, 189.9, 0.0], 'cumulative_reward': -25.293, 'done': False}
{'state': [3.27, 3.